
모델 저장 및 불러오기: [link](https://pytorch.org/tutorials/beginner/saving_loading_models.html)
=========================
파이토치 모델의 저장 및 불러오는 것에 대한 다양한 유스케이스에 대해서 살펴봅니다

우선 아래 3가지 핵심 함수를 알아야합니다.
* [torch.save](https://pytorch.org/docs/stable/torch.html?highlight=save#torch.save): serialize한 객체를 디스크에 저장합니다. 이 함수는 serialize를 위해 Python의 pickle 유틸리티를 사용합니다. 이 기능을 사용하여 모든 종류의 객체의 모델, 텐서 및 사전을 저장할 수 있습니다.

* [torch.load](https://pytorch.org/docs/stable/torch.html?highlight=torch%20load#torch.load): pickle의 unpickling 기능을 사용하여 피클 링 된 오브젝트 파일을 메모리로 deserialize합니다. 이 기능은 또한 데이터를 device에 불러올 때도 사용합니다 ([참고 device에 모델 저장 및 불러오기](https://pytorch.org/tutorials/beginner/saving_loading_models.html#saving-loading-model-across-devices)).

* [torch.nn.Module.load_state_dict](https://pytorch.org/docs/stable/nn.html?highlight=load_state_dict#torch.nn.Module.load_state_dict): deserialize된 state_dict를 사용하여 모델의 파라미터를 로드합니다. ([참고 sta_dict이란](https://pytorch.org/tutorials/beginner/saving_loading_models.html#what-is-a-state-dict))

``state_dict``이란?
------------------------

PyTorch에서 ``torch.nn.Module`` 모델의 학습 가능한 파라미터들은(i.e. weights 및 biases) 모델의 파라미터에 (``model.parameters()``로 접근되는) 포함되어 있습니다. state_dict은 간단히 말해 각 레이어를 파라미터 텐서로 매핑하는 Python 사전 객체입니다. 오직 학습가능한 파라미터(conv layers, linear layers 등)가있는 레이어와 등록된 버퍼들(batchnorm의 running_mean)만 state_dict이 있다는 것을 유의해야합니다. Optimizer 객체(``torch.optim``)들에도 Optimizer 상태뿐만 아니라 사용된 하이퍼파라미터 정보가 포함된 state_dict가 있습니다.

state_dict 객체는 Python 사전이므로 쉽게 저장, 업데이트, 변경 및 복원 할 수 있게 하며, PyTorch 모델과 Optimizer에 엄청난 모듈성(modularity)을 제공합니다.

### Example:
분류기(Classifier) 학습하기 튜토리얼에서 사용한 간단한 모델의 state_dict 를 살펴보도록 하겠습니다.

In [2]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# 모델 정의
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 모델 초기화
model = TheModelClass()

# 옵티마이저 초기화
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 모델의 state_dict 출력
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# 옵티마이저의 state_dict 출력
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
conv1.weight 	 torch.Size([6, 3, 5, 5])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([16, 6, 5, 5])
conv2.bias 	 torch.Size([16])
fc1.weight 	 torch.Size([120, 400])
fc1.bias 	 torch.Size([120])
fc2.weight 	 torch.Size([84, 120])
fc2.bias 	 torch.Size([84])
fc3.weight 	 torch.Size([10, 84])
fc3.bias 	 torch.Size([10])
Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [140107487062560, 140107487062344, 140107487062416, 140107486598560, 140107486598488, 140107486598632, 140107486598704, 140107486598776, 140107486598848, 140107486598920]}]


추론(Inference)을 위한 모델 저장 및 불러오기
------------------------------------

### ``state_dict`` 저장 및 불러오기 (권장하는 방법)

In [ ]:
#Save:
torch.save(model.state_dict(), PATH)

In [ ]:
#Load:
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.eval()

추론을 위해 모델을 저장할 때, 모델의 학습된 파라미터들만 저장하면 됩니다. `torch.save()` 함수를 사용하여 모델의 state_dict를 저장하면 나중에 모델 복원에 대한 유연성이 높아 지므로 state_dict를 사용하여 모델을 저장하는게 권장되는 방법입니다.

보통 PyTorch는 `.py`나 `.pth` 확장자로 저장합니다.

추론을 실행하기 전에는 반드시 `model.eval()` 을 호출하여 드롭아웃 및 배치 정규화를 평가 모드로 설정하여야 합니다. 이것을 하지 않으면 추론 결과가 일관성 없게 출력됩니다.

`load_state_dict()` 함수는 저장된 객체의 경로가 아닌, 사전 객체를 전달해야함을 유의해야합니다. 즉 `load_state_dict()` 함수에 저장된 state_dict를 전달하기 전에 반드시 dict을 deserialize해야합니다. 예를 들어, `model.load_state_dict(PATH)`를 사용하여 로드할 수 없습니다.

### 추론 / 학습 재게를 위한 일반 체크포인트 저장 및 불러오기

In [ ]:
# 저장:
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            ...
            }, PATH)

In [ ]:
# 불러오기:
model = TheModelClass(*args, **kwargs)
optimizer = TheOptimizerClass(*args, **kwargs)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - or -
model.train()

inference 또는 training 재개를 위해 일반적인 체크 포인트를 저장할 때 모델의 state_dict뿐만아니라 다른 것들도 저장해야합니다. (ex. optimizer의 state_dict) 모델이 학습을 하며 갱신되는 버퍼와 매개변수가 포함된 옵티마이저의 state_dict 도 함께 저장하는 것이 중요합니다. 이외로 마지막 epoch, 최근에 기록 된 훈련 손실, 외부 `torch.nn.Embedding layer` 등을 함께 저장할 수 있습니다.

이러한 여러 구성 요소를 함께 저장하려면 이들을 python dict에 넣고 `torch.save()`로 이들을 serialize하면 됩니다. 일반적으로 PyTorch는 `.tar` 파일 확장명을 사용하여 이러한 check point들을 저장합니다.

이들을 로드하려면 먼저 모델과 optimizer를 초기화 한 다음 `torch.load()`를 사용하여 사전을 불러옵니다. 이제 여기에서 사전을 쿼리하여 저장된 항목에 쉽게 접근 할 수 있습니다.

추론 전에 평가를 위한 dropout과 batch noralization layer를 설정하기 위해 반드시 `model.eval()`를 호출해야함을 유의해야합니다. 이렇게하지 않으면 추론 결과가 일관성 없게 출력 됩니다. 만약에 학습을 계속하고 싶다면 `model.train()`을 호출하여 학습 모드로 전환되도록 해야 합니다

### 여러 개의 모델을 하나의 파일에 저장하기

In [ ]:
#Save
torch.save({
            'modelA_state_dict': modelA.state_dict(),
            'modelB_state_dict': modelB.state_dict(),
            'optimizerA_state_dict': optimizerA.state_dict(),
            'optimizerB_state_dict': optimizerB.state_dict(),
            ...
            }, PATH)

In [ ]:
#Load
modelA = TheModelAClass(*args, **kwargs)
modelB = TheModelBClass(*args, **kwargs)
optimizerA = TheOptimizerAClass(*args, **kwargs)
optimizerB = TheOptimizerBClass(*args, **kwargs)

checkpoint = torch.load(PATH)
modelA.load_state_dict(checkpoint['modelA_state_dict'])
modelB.load_state_dict(checkpoint['modelB_state_dict'])
optimizerA.load_state_dict(checkpoint['optimizerA_state_dict'])
optimizerB.load_state_dict(checkpoint['optimizerB_state_dict'])

modelA.eval()
modelB.eval()
# - or -
modelA.train()
modelB.train()

GAN이나 seq2seq, 앙상블 모델과 같이 여러개의 `torch.nn.Modules1로 이루어진 모델을 저장할 때는 일반적인 체크포인트를 저장하는 방법과 같은 방식으로 하면 됩니다. 다시말해, 각 모델의 state_dict과 옵티마이저를 사전으로 저장하면 됩니다. 앞에 언급한것과 같이 학습을 재개하는데 필요한 다른 항목들을 사전에 추가하여 저장할 수 있습니다.

일반적인 PyTorch 사용 규칙은 `.tar` 확장자를 사용하여 체크포인트를 저장하는 것입니다.

항목들을 불러올 때에는 먼저 모델과 옵티마이저를 초기화한 후, `torch.load()` 를 사용하여 사전을 불러옵니다. 이후로는 저장된 항목들을 사전에 원하는대로 사전에 질의하여 쉽게 접근할 수 있습니다.

추론을 실행하기 전에는 반드시 `model.eval()` 을 호출하여 드롭아웃 및 배치 정규화를 평가 모드로 설정하여야 합니다. 이것을 하지 않으면 추론 결과가 일관성 없게 출력됩니다. 만약 학습을 계속하고 싶다면, `model.train()` 을 호출하여 학습 모드로 설정해야 합니다.

### 다른 모델의 매개변수를 사용하여 빠르게 모델 시작하기(warmstart)

In [ ]:
#Save
torch.save(modelA.state_dict(), PATH)

In [ ]:
#Load
modelB = TheModelBClass(*args, **kwargs)
modelB.load_state_dict(torch.load(PATH), strict=False)

부분적으로 모델을 불러오거나, 모델의 일부를 불러오는 것은 전이학습 또는 새로운 복잡한 모델을 학습할 때 일반적인 시나리오입니다. 학습된 매개변수를 사용하면, 일부만 사용한다 하더라도 학습 과정을 빠르게 시작할 수 있고, 처음부터 시작하는 것보다 훨씬 빠르게 모델이 수렴하도록 도울 것입니다.

몇몇 키를 제외하고 state_dict 의 일부를 불러오거나, 적재하려는 모델보다 더 많은 키를 갖고 있는 state_dict 를 불러올 때에는 `load_state_dict()` 함수에서 `strict` 인자를 False 로 설정하여 일치하지 않는 키들을 무시하도록 해야 합니다.

한 계층에서 다른 계층으로 매개변수를 불러오고 싶지만, 일부 키가 일치하지 않을 때에는 적재하려는 모델의 키와 일치하도록 state_dict 의 매개변수 키의 이름을 변경하면 됩니다.

Device에 모델 저장 및 불러오기
-------------------------------------

### GPU에서 저장하고 CPU에서 불러오기

In [ ]:
torch.save(model.state_dict(), PATH)

In [ ]:
device = torch.device("cuda")
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.to(device)
# 모델에서 사용하는 input Tensor들은 input = input.to(device) 을 호출해야 합니다.

GPU에서 학습한 모델을 GPU에서 불러올 때는 `torch.load()` 함수의 `map_location` 인자에 `torch.device('cpu')`을 전달합니다. 이 경우에는 Tensor에 저장된 내용들은 `map_location` 인자를 사용하여 CPU 장치에 동적으로 재배치됩니다.

### GPU에서 저장하고 GPU에서 불러오기

In [ ]:
torch.save(model.state_dict(), PATH)

In [ ]:
device = torch.device("cuda")
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.to(device)
# 모델에서 사용하는 input Tensor들은 input = input.to(device) 을 호출해야 합니다.

GPU에서 학습한 모델을 GPU에서 불러올 때에는, 초기화된 `model`에 `model.to(torch.device('cuda'))`을 호출하여 CUDA 최적화된 모델로 변환해야 합니다. 또한, 모델에 데이터를 제공하는 모든 입력에 `.to(torch.device('cuda'))` 함수를 호출해야 합니다. `my_tensor.to(device)`를 호출하면 GPU에 `my_tensor` 의 복사본을 반환하기 때문에, Tensor를 직접 덮어써야 합니다: `my_tensor = my_tensor.to(torch.device('cuda'))` 

### CPU에서 저장하고 GPU에서 불러오기

In [ ]:
torch.save(model.state_dict(), PATH)

In [ ]:
device = torch.device("cuda")
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location="cuda:0"))  # 사용할 GPU 장치 번호를 선택합니다.
model.to(device)
# 모델에서 사용하는 input Tensor들은 input = input.to(device) 을 호출해야 합니다.

CPU에서 학습한 모델을 GPU에서 불러올 때는 `torch.load()` 함수의 `map_location` 인자에 cuda:device_id 을 설정합니다. 이렇게 하면 모델이 해당 GPU 장치에 불러와집니다. 다음으로 `model.to(torch.device('cuda'))`을 호출하여 모델의 매개변수 Tensor들을 CUDA Tensor들로 변환해야 합니다. 마지막으로 모든 모델 입력에 `.to(torch.device('cuda'))`을 사용하여 CUDA 최적화된 모델을 위한 데이터로 만들어야 합니다. `my_tensor.to(device)`를 호출하면 GPU에 my_tensor 의 복사본을 반환합니다. 이 동작은 `my_tensor`를 덮어쓰지 않기 때문에, Tensor를 직접 덮어써야 합니다: `my_tensor = my_tensor.to(torch.device('cuda'))`

## `torch.nn.DataParallel` 모델 저장하기

In [ ]:
torch.save(model.module.state_dict(), PATH)

In [7]:
# 사용할 장치에 불러옵니다.

`torch.nn.DataParallel`은 병렬 GPU 활용을 가능하게 하는 모델 wrapper 입니다. 일반적으로 `DataParallel` 모델을 저장하려면  `model.module.state_dict()`을 저장하면 됩니다. 이 방법을 통해 원하는 어떤 device에 원하는 어떤 방법으로 모델을 로드할 수있는 유용성을 줍니다.